In [1]:
import time
from tqdm import tqdm
import pandas as pd
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import chromedriver_binary

In [2]:
link = 'https://stock-pictures.netlify.app'

In [3]:
service = Service(chromedriver_binary.chromedriver_filename)

driver = webdriver.Chrome(service = service)
driver.get(link)

## 1) Scrolling to reach the buttom

In [6]:
for i in tqdm(range(0,5000000,1000)):
    
    driver.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [10:19<00:00,  8.08it/s]


## 2) Scraping the Page Course

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

## 3) Checking number of images that has to be scraped

In [9]:
len(soup.find_all('div',class_ = 'container'))

9104

## 4) Scraping the Image Details

In [10]:
data = []

for sp in tqdm(soup.find_all('div',class_ = 'container')):
    image_link = sp.find('img').get('src')
    tags = sp.find('div',class_ = 'tags').text[7:].split(',')
    likes = sp.find('div',class_ = 'likes-comments').find_all('span')[0].text[:-6]
    comments = sp.find('div',class_ = 'likes-comments').find_all('span')[1].text[:-10]

    data.append([image_link,' '.join(tags),likes,comments])

100%|███████████████████████████████████████████████████████████████████████████| 9104/9104 [00:00<00:00, 10535.37it/s]


## 5) Saving data in a DataFrame

In [11]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,Clouds Sky Atmosphere Blue Sky,196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,Bird Ornithology Hummingbird,76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,Sea Rainbow Rainfall Subtropical,282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,Cherry Blossoms Road Japan Sakura,42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,Cape Marguerite Flower Plant,39,15


In [12]:
df.shape

(9104, 4)

## 6) Checking Null Values

In [13]:
df.isnull().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

## 7) Checking Duplicate Values

In [14]:
df['img_link'].describe()

count                                                  9104
unique                                                 9088
top       https://cdn.pixabay.com/photo/2022/03/06/05/30...
freq                                                      2
Name: img_link, dtype: object

## 8) Removing Duplicate Values

In [15]:
df = df.drop_duplicates()

## 9) Saving the DataFrame as CSV

In [16]:
df.to_csv('images.csv')